In [3]:
import io
import pickle
import pandas as pd

path = 'D:\measure.gtm'
path ='C:/Users/user/kriolab2.gtm'
path2 = 'C:/Users/user/measure.gtm'

In [6]:
### FINAL NOTEBOOK
# %%time
import sys
import os
import pandas as pd
from importlib import reload
import struct
# import lib_conv_csv_gtm as lib
import numpy as np

zero_level_csv_path = "D:/WORK/For_Convertor/"
zero_level_csv_path = 'C:\Users\user\Documents\m_project\2022 Осень\Приборы\1151\'
list_of_zero_level_csv_path = os.listdir(zero_level_csv_path) # 

###
list_of_first_level_csv_path = []
for i1 in list_of_zero_level_csv_path:
    list_of_first_level_csv_path.append(zero_level_csv_path + i1 + '/')
###
# print(list_of_first_level_csv_path)
###
for i1 in list_of_first_level_csv_path:
    
    list_of_end_level_csv_files = []
    list_of_end_level_csv_files = os.listdir(i1)
    
    ### получаем обрабатываемые пути файлов csv
    list_of_csv_path = []
    for i2 in list_of_end_level_csv_files:
        if '.csv' in i2:
            list_of_csv_path.append(i1+i2)
    
    ### начинаем обработку файлов csv и gtm   
    for csv_file in list_of_csv_path:        
        # открываем и читаем датафрейм от csv файла
        df_of_csv_file = pd.read_csv(csv_file, encoding='cp1251', sep = ';', decimal=',')
        df_of_csv_file.rename(columns ={'объект' : 'скважина'}, inplace = True) # переименовываем
        actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                      '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0', 'дата время', 'коса', 'скважина']
        #добавляем новые индексы, чтобы использовать 15м косы
        df_of_csv_file = df_of_csv_file.reindex(columns = actual_columns)
        df_of_csv_file = df_of_csv_file[actual_columns] # на всякий
        df_of_csv_file['дата время'] = pd.to_datetime(df_of_csv_file['дата время'], dayfirst=True)

        #GTM FILE
        path = csv_file[:-4] + '.gtm' # обозначаем файл gtm на основе файла csv
        ### open and create gpm file
        file = open(path, 'wb')
        file.write(b'GTM0')
        
#         continue #
        for i in range (0, len(df_of_csv_file)): # !подставить len(df_of_raw_data_files) вместо 5
        #     wtite_to_gtm_file(path = path_of_gtm_file)
            count_censors = 16
#             file = open(path, 'ab') #

            # первый байт скорее зависит от кол-ва символов в строке, \x63 для 3 в строке
            # блок размера записи
            name_object = str(df_of_csv_file.iloc[i]['скважина'])
            len_of_well_name = len(name_object)

            #
            more_sensors = 3
            if np.isnan(df_of_csv_file.iloc[i]['12,0']): #какой пиздец и костыли
                more_sensors = 0
            count_censors = count_censors+more_sensors
            file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5)) #длинна записи, хз как считается 
            ###


            file.write(b'\x54\x30\x32') # добавляет T02 не знаю что это

            file.write(struct.pack('b', count_censors)) # кол-во датчиков в косе
            ###
# #########################
#             file.write(b'\x00') # cекунды, по умолчанию 0, если больше или равно \x1e то обнуляет
#             ###
#             ###
#             file.write(b'\x00') # минуты*8  хзхзхзхзхз
#             ###
#             ###
#             month = df_of_csv_file.iloc[i]['дата время'].month
            
#             day = df_of_csv_file.iloc[i]['дата время'].day  
#             file.write(struct.pack('B', day+32*month)) # дата от 01.01.1980
#             ###
            
#             ##### Блок написания года
            
#             year = df_of_csv_file.iloc[i]['дата время'].year-1980
#             file.write(struct.pack('B', year*2))
#         #     file.write(b'\x00') # год + 1980, значения по пол года (ну не совсем так)
#             ####
#         #     struct.pack('>h',  df_of_raw_data_files.iloc[i]['скважина'])
#         ###########################
        
            ####################### блок времени и даты
            ##### блок написания времени
            minute = df_of_csv_file.iloc[i]['дата время'].minute
            second = df_of_csv_file.iloc[i]['дата время'].second
            file.write(struct.pack('B', second + (minute%8)*32)) # cекунды и остаток минут деленные на 8
            hour = df_of_csv_file.iloc[i]['дата время'].hour
            file.write(struct.pack('B', hour*8 + (minute//8))) # час и минута деленная на 8

            ##### блок написания даты
            month = df_of_csv_file.iloc[i]['дата время'].month
            day = df_of_csv_file.iloc[i]['дата время'].day  
            file.write(struct.pack('B', day+32*(month%8))) # дата от 01.01.1980
            ## Блок написания года 
            year = df_of_csv_file.iloc[i]['дата время'].year-1980
            polovina_goda=0
            if month >=8:
                polovina_goda=1
            file.write(struct.pack('B', year*2+polovina_goda))
            ###########################
            ###########################

            ### блок запосления номера косы  не могу правильность проверить
        #     file.write(b'\x64\x00') # , номер косы
            file.write(struct.pack('>h', df_of_csv_file.iloc[i]['коса']))
            ###
            file.write(b'\x00\x00') # , превышение уровня, у нас его нет

            ### блок названия объекта
        #     file.write(b'\x04HUU\x00') # название объекта - \x длинна названия+1, название, пустой байт
        #     name_object = str(df_of_raw_data_files.iloc[i]['скважина'])
        #     if len(name_object) <3:
        #         name_object = ' '*(3-len(name_object)) + name_object

        #     file.write(b'\x04') # длина имени +1
            file.write(struct.pack('B', len_of_well_name+1))
            file.write(bytes(name_object, 'utf-8'))
            file.write(b'\x00')
            ###

            ###блок написания температур
            actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                              '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0']
            offset_sensor = [0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 20, 20, 10]
        #     actual_columns = ['0,0'] # временно
            for index, j in enumerate(actual_columns):
                if str(df_of_csv_file.iloc[i][j]) != 'nan': # проверка на nan
                    current_temperature_number = float(df_of_csv_file.iloc[i][j])
                    integer_part = int(abs(current_temperature_number)//1)*100
                    decimal_part = int(round(abs(current_temperature_number)*100, 0)%100)
                    number = integer_part+decimal_part
                    if current_temperature_number<0:
                        number = number*(-1)
                    file.write(struct.pack('>h', number))
                    file.write(b'\xfe\n') #пишем степень и знак цельсия
                    file.write(struct.pack('>B', offset_sensor[index]))

            ### последний байт, длинна строки
            file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5))
        file.close()
#     print (list_of_csv_path)
print('end')

SyntaxError: EOL while scanning string literal (Temp/ipykernel_8768/369196860.py, line 12)

In [17]:
int(round(abs(float(df_of_csv_file.iloc[i][j]))*100, 0)%100)

0

In [9]:
j

'0,0'

In [7]:
df_of_csv_file

,"0,0","0,5","1,0","1,5","2,0","2,5","3,0","3,5","4,0","4,5",...,"7,0","8,0","9,0","10,0","12,0","14,0","15,0",дата время,коса,скважина
0,-12,-14.5,-9.6,-14.3,-14.6,-14,-14,-13.6,-13.5,-13,...,-12.2,-10.1,-10.8,-11.9,NaN,NaN,NaN,2021-04-03 15:15:00,20434,3
1,-16.7,-17.8,-16.3,-14.8,-13.4,-12.3,-11,-10,-9.1,-8.5,...,-5.7,-5.1,-4.7,-4.5,NaN,NaN,NaN,2021-04-03 17:26:00,20434,3


In [200]:
import sys
import os
import pandas as pd
from importlib import reload
import struct
import lib_conv_csv_gtm as lib
import numpy as np
reload(lib)

folder_csv_path = "D:/WORK/For_Convertor/PSP" # папка с csvшками
path_of_gtm_file = folder_csv_path + '/PS.gtm'
path_of_gtm_file = 'C:/Users/user/1.gtm' #временно

# df_of_raw_data_files = lib.get_list_of_csv_file_data(folder_csv_path)
# lists_of_csv_file_data = lib.get_list_of_csv_file_data(folder_csv_path)
csv_file = folder_csv_path + '/06-05-2021.csv'
df_of_csv_file = pd.read_csv(csv_file, encoding='cp1251', sep = ';', decimal=',')
df_of_csv_file.rename(columns ={'объект' : 'скважина'}, inplace = True) # переименовываем
actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                      '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0', 'дата время', 'коса', 'скважина']
        #добавляем новые индексы, чтобы использовать 15м косы
df_of_csv_file = df_of_csv_file.reindex(columns = actual_columns)
df_of_csv_file = df_of_csv_file[actual_columns] # на всякий
df_of_csv_file['дата время'] = pd.to_datetime(df_of_csv_file['дата время'], dayfirst=True)
df_of_raw_data_files = df_of_csv_file
#временно
lib.open_and_close_gtm_file(path = path_of_gtm_file)


for i in range (0, len(df_of_raw_data_files)): # !подставить len(df_of_raw_data_files) вместо 5
#     wtite_to_gtm_file(path = path_of_gtm_file)
    count_censors = 16
#     if
    file = open(path_of_gtm_file, 'ab')

    # первый байт скорее зависит от кол-ва символов в строке, \x63 для 3 в строке
    # блок размера записи
    name_object = str(df_of_raw_data_files.iloc[i]['скважина'])
    len_of_well_name = len(name_object)
    
    #
    more_sensors = 3
    if np.isnan(df_of_raw_data_files.iloc[i]['12,0']): #какой пиздец и костыли
        more_sensors = 0
    count_censors = count_censors+more_sensors
    file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5)) #длинна записи, хз как считается 
    ###
    
    
    file.write(b'\x54\x30\x32') # добавляет T02 не знаю что это
    
    file.write(struct.pack('b', count_censors)) # кол-во датчиков в косе
    ####################### блок времени и даты
    ##### блок написания времени
    minute = df_of_raw_data_files.iloc[i]['дата время'].minute
    second = df_of_raw_data_files.iloc[i]['дата время'].second
    file.write(struct.pack('B', second + (minute%8)*32)) # cекунды и остаток минут деленные на 8
    hour = df_of_raw_data_files.iloc[i]['дата время'].hour
    file.write(struct.pack('B', hour*8 + (minute//8))) # час и минута деленная на 8

    ##### блок написания даты
    month = df_of_raw_data_files.iloc[i]['дата время'].month
    day = df_of_raw_data_files.iloc[i]['дата время'].day  
    file.write(struct.pack('B', day+32*(month%8))) # дата от 01.01.1980
    ## Блок написания года 
    year = df_of_raw_data_files.iloc[i]['дата время'].year-1980
    polovina_goda=0
    if month >=8:
        polovina_goda=1
    file.write(struct.pack('B', year*2+polovina_goda))
    ###########################
    
    # напр: если хотим прибавить 20 лет, то прибавляем 40 в десятичной -> ппереводим в двоичную
#     struct.pack('>h',  df_of_raw_data_files.iloc[i]['скважина'])

    ### блок запосления номера косы  не могу правильность проверить
#     file.write(b'\x64\x00') # , номер косы
    file.write(struct.pack('>h', df_of_raw_data_files.iloc[i]['коса']))
    ###
    file.write(b'\x00\x00') # , превышение уровня, у нас его нет
    
    ### блок названия объекта
#     file.write(b'\x04HUU\x00') # название объекта - \x длинна названия+1, название, пустой байт
#     name_object = str(df_of_raw_data_files.iloc[i]['скважина'])
#     if len(name_object) <3:
#         name_object = ' '*(3-len(name_object)) + name_object
    
#     file.write(b'\x04') # длина имени +1
    file.write(struct.pack('B', len_of_well_name+1))
    file.write(bytes(name_object, 'utf-8'))
    file.write(b'\x00')
    ###
    
    ###блок написания температур
    actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                      '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0']
    offset_sensor = [0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 20, 20, 1]
#     actual_columns = ['0,0'] # временно
    for index, j in enumerate(actual_columns):
        if str(df_of_raw_data_files.iloc[i][j]) != 'nan': # проверка на nan
            integer_part = int(abs(df_of_raw_data_files.iloc[i][j])//1)*100
            decimal_part = int(round(abs(df_of_raw_data_files.iloc[i][j])*100, 0)%100)
            number = integer_part+decimal_part
            if df_of_raw_data_files.iloc[i][j]<0:
                number = number*(-1)
            file.write(struct.pack('>h', number))
            file.write(b'\xfe\n') #пишем степень и знак цельсия
            file.write(struct.pack('>B', offset_sensor[index]))

    ### последний байт, длинна строки
    file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5))
    file.close()
print('ВСЁ')

ВСЁ


In [138]:
polovina_goda

0

In [72]:

# df_of_csv_file
csv_file

'D:/WORK/For_Convertor/Nap_CPS_PSP/19-04-2021.csv'

In [596]:
df_of_raw_data_files[df_of_raw_data_files['скважина'] == 5411]

,"0,0","0,5","1,0","1,5","2,0","2,5","3,0","3,5","4,0","4,5",...,"7,0","8,0","9,0","10,0","12,0","14,0","15,0",дата время,коса,скважина
0,0.33,-0.01,0.89,0.22,1.23,1.31,0.97,1.63,0.98,1.17,...,1.40,0.53,0.74,0.95,NaN,NaN,NaN,2021-04-23 11:28:00,20417,5411
20,-2.59,-4.52,-2.67,-2.01,-1.38,-0.90,-0.61,-0.53,-0.48,-0.48,...,-0.89,-1.05,-1.29,-1.46,NaN,NaN,NaN,2021-04-23 13:55:00,20417,5411


In [450]:
struct.unpack('B', b'\x14')

(20,)

In [2]:
file = open (path, 'rb', buffering = 0)
file2 = open(path2, 'wb')

df = pd.DataFrame(columns=['bite', 'bin_ord', 'ord'])

# text2 = b'GTM0\x00\x16T02\x01H\xafxP\x03\t\x00\x00\x02A\x00\n\x7f\xfe\n\n\x00\x17'
# text = file.read(105)
# file2.write(text2)

# size_of_file = len(text)
# text = file.read(1)

# while text != b'':
# #     print(text)
# #     print (text, '\t', ord(text), '\t', bin(ord(text)), '\t')
#     df = df.append({'bite' : text,
#                    'bin_ord' :  ord(text),
#                    'ord' : bin(ord(text)),
#                    'chr_ord' : chr(ord(text))
#                    }, ignore_index=True)
#     text = file.read(1)

# #     print(text)

# file2.write(hex(np.float16(2.3).tostring()[0]))           
# file2.write(hex(np.float16(2.3).tostring()[1]))
file2.write(b'GTM0\x00\x63\x54\x30\x32\x10\x00\x00\x21\x00\x02\x9a\x00\x00\x04HUU\x00') # записали дату, косу и название объекта
#cT02  \x63\x54\x30\x32

#RT00 \x52\x54\x30\x30
file2.write(b'\x01\x3a\xfe\n\x00')
file2.write(b'\x01\x3b\xfe\n\x05')
file2.write(b'\x01\x3c\xfe\n\x05')
file2.write(b'\x01\x3d\xfe\n\x05')
file2.write(b'\x01\x3e\xfe\n\x05')
file2.write(b'\x01\x3f\xfe\n\x05')
file2.write(b'\x01\x40\xfe\n\x05')
file2.write(b'\x01\x41\xfe\n\x05')
file2.write(b'\x01\x42\xfe\n\x05')
file2.write(b'\x01\x43\xfe\n\x05')
file2.write(b'\x01\x44\xfe\n\x05')
file2.write(b'\x01\x45\xfe\n\n')
file2.write(b'\x01\x46\xfe\n\n')
file2.write(b'\x01\x47\xfe\n\n')
file2.write(b'\x01\x48\xfe\n\n')
file2.write(b'\x01\x49\xfe\n\n')
file2.write(b'\x00c')

file2.write(b'\x00cT02\x10\x00\x00\x21\x00\x02\x9b\x00\x00\x04HUU\x00') # записали дату, косу и название объекта
file2.write(b'\x01\x3a\xfe\n\x00')
file2.write(b'\x01\x3b\xfe\n\x05')
file2.write(b'\x01\x3c\xfe\n\x05')
file2.write(b'\x01\x3d\xfe\n\x05')
file2.write(b'\x01\x3e\xfe\n\x05')
file2.write(b'\x01\x3f\xfe\n\x05')
file2.write(b'\x01\x40\xfe\n\x05')
file2.write(b'\x01\x41\xfe\n\x05')
file2.write(b'\x01\x42\xfe\n\x05')
file2.write(b'\x01\x43\xfe\n\x05')
file2.write(b'\x01\x44\xfe\n\x05')
file2.write(b'\x01\x45\xfe\n\n')
file2.write(b'\x01\x46\xfe\n\n')
file2.write(b'\x01\x47\xfe\n\n')
file2.write(b'\x01\x48\xfe\n\n')
file2.write(b'\x01\x49\xfe\n\n')
file2.write(b'\x00c')
# file2.write(b'\x00cT02\x10\xc1YqRO\xc8\x00\x00\x04Hui\x00\n"\xfe\n\x00\t\xdc\xfe\n\x05\t\xd4\xfe\n\x05\t\xe5\xfe\n\x05\t`\xfe\n\x05\t\xcc\xfe\n\x05\t\x8d\xfe\n\x05\t\xc5\xfe\n\x05\t\xaf\xfe\n\x05\t\xbb\xfe\n\x05\n*\xfe\n\x05\nQ\xfe\n\n\n\x97\xfe\n\n\to\xfe\n\n\n\xa1\xfe\n\n\t\xae\xfe\n\n\x00c\x00cT02\x10\xdbYqRO\xc8\x00\x00\x04Hiy\x00\n<\xfe\n\x00\t\xf5\xfe\n\x05\t\xf4\xfe\n\x05\t\xfe\xfe\n\x05\t\x7f\xfe\n\x05\t\xe5\xfe\n\x05\t\xa7\xfe\n\x05\t\xde\xfe\n\x05\x00\x00\xfe\x00\x05\x00\x00\xfe\x00\x05\x00\x00\xfe\x00\x05\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00c\x00cT02\x10#[qRO\xc8\x00\x00\x04hhh\x00\t\xe2\xfe\n\x00\t\xc2\xfe\n\x05\t\xba\xfe\n\x05\t\x93\xfe\n\x05\t\xb8\xfe\n\x05\t\xba\xfe\n\x05\t\x86\xfe\n\x05\t\xb3\xfe\n\x05\t\xc2\xfe\n\x05\t\xb5\xfe\n\x05\t\xde\xfe\n\x05\t\xeb\xfe\n\n\t\xec\xfe\n\n\t\x9c\xfe\n\n\t\xdc\xfe\n\n\t\x88\xfe\n\n\x00c\x00cT02\x107[qRO\xc8\x00\x00\x04HHH\x00\t\xdc\xfe\n\x00\t\xbc\xfe\n\x05\t\xae\xfe\n\x05\t\x8c\xfe\n\x05\t\xb2\xfe\n\x05\t\xad\xfe\n\x05\t\x80\xfe\n\x05\t\xac\xfe\n\x05\t\xbc\xfe\n\x05\t\xaf\xfe\n\x05\t\xd1\xfe\n\x05\t\xde\xfe\n\n\t\xd9\xfe\n\n\t\x96\xfe\n\n\t\xd0\xfe\n\n\t\x82\xfe\n\n\x00c\x00cT02\x10\x04\\qRO\xc8\x00\x00\x04HhH\x00\t\x9d\xfe\n\x00\tq\xfe\n\x05\t\x82\xfe\n\x05\tS\xfe\n\x05\t\x85\xfe\n\x05\tg\xfe\n\x05\t[\xfe\n\x05\tg\xfe\n\x05\t\x8a\xfe\n\x05\to\xfe\n\x05\t\x92\xfe\n\x05\t\xa0\xfe\n\n\t\x8d\xfe\n\n\tu\xfe\n\n\t\x83\xfe\n\n\tV\xfe\n\n\x00c\x00cT02\x10\x04dqRO\xc8\x00\x00\x04hHh\x00\x08\xf8\xfe\n\x00\t\x00\xfe\n\x05\x08\xf7\xfe\n\x05\t\r\xfe\n\x05\x08\xfb\xfe\n\x05\t\t\xfe\n\x05\x08\xfc\xfe\n\x05\t\t\xfe\n\x05\t\x05\xfe\n\x05\t\n\xfe\n\x05\x08\xfa\xfe\n\x05\x08\xfa\xfe\n\n\t\x03\xfe\n\n\x08\xfe\xfe\n\n\x08\xf7\xfe\n\n\x08\xd7\xfe\n\n\x00c\x00cT02\x10\x04lqRO\xc8\x00\x00\x04HAA\x00\x08\xec\xfe\n\x00\x08\xf9\xfe\n\x05\x08\xea\xfe\n\x05\x08\xfb\xfe\n\x05\x08\xee\xfe\n\x05\t\x02\xfe\n\x05\x08\xf6\xfe\n\x05\x08\xf5\xfe\n\x05\x08\xf8\xfe\n\x05\t\x04\xfe\n\x05\x08\xed\xfe\n\x05\x08\xee\xfe\n\n\x08\xf6\xfe\n\n\x08\xf1\xfe\n\n\x08\xeb\xfe\n\n\x08\xc5\xfe\n\n\x00c\x00cT02\x10\x04tqRO\xc8\x00\x00\x04HAH\x00\x08\'\xfe\n\x00\x08/\xfe\n\x05\x08%\xfe\n\x05\x086\xfe\n\x05\x08*\xfe\n\x05\x08E\xfe\n\x05\x082\xfe\n\x05\x08,\xfe\n\x05\x08.\xfe\n\x05\x08@\xfe\n\x05\x08(\xfe\n\x05\x08#\xfe\n\n\x08+\xfe\n\n\x08.\xfe\n\n\x08&\xfe\n\n\x08!\xfe\n\n\x00c\x00cT02\x10\x04|qRO\xc8\x00\x00\x04AHA\x00\x08`\xfe\n\x00\x08\xad\xfe\n\x05\x08e\xfe\n\x05\x08\xb5\xfe\n\x05\x08o\xfe\n\x05\x08\xbd\xfe\n\x05\x08x\xfe\n\x05\x08\xaa\xfe\n\x05\x08z\xfe\n\x05\x08\xbe\xfe\n\x05\x08t\xfe\n\x05\x08b\xfe\n\n\x08w\xfe\n\n\x08m\xfe\n\n\x08r\xfe\n\n\x08S\xfe\n\n\x00c')
#\x48\x55\x55  HUU
####
# (b'GTM0 (вступление) 
# \x00cT02  (константа) 
# \x10 (кол-во датчиков)  
# \x13Y #секунды минуты qRO
# \xc8 (меняет номер прибора) 
# \x00 (влияет на смещение косы) \x00  (добавляет метры(смещение косы)) 
# \x04HUI (длинна ячейки(не трогать, потом подобрать для четырехточеского) и название объекта\
# \x00
# \t+\xfe (меняет температуру) !!!!!!можно заменить на \t\x2b\xfe
## \x00\x00\x00\xfe\ 0 по цельсию

# \n\x00\x08\xe7\xfe\n
# \x05 (смещение косы (плюс 0.5))   \x08\xf0\xfe\n\x05\t\x07\xfe\n\x05\x08\x82\xfe\n\x05\x08\xf0\xfe\n\x05\x08\xd0\xfe\n\x05\x08\xe2\xfe\n\x05\x08\xd2\xfe\n\x05\x08\xe5\xfe\n\x05\t?\xfe\n\x05\tf\xfe\n\n\t\x8d\xfe\n\n\x08\xb3\xfe\n\n\t\xb6\xfe\n\n\x08\xe5\xfe\n\n\x00c\
####

file.close()
file2.close()
# df.to_excel(path+'.xlsx')
print(df.shape)

(0, 3)


In [359]:
('a').to_bytes(3, byteorder='big')

AttributeError: 'str' object has no attribute 'to_bytes'

In [52]:
len_of_well_name = 3
type(len_of_well_name)

int

In [4]:
# Text to Binary
message = "Hello"  # str
binary_message = message.encode('utf-8')
print(type(binary_message))  # bytes

import codecs
cipher_text = codecs.encode(message, 'rot_13')
print(cipher_text)

<class 'bytes'>
Uryyb


In [25]:
import sys
sys.path.insert(0, "C:/Users/user/Documents/WORK/Library_and_other")
import lib_conv_csv_gtm as lib

lib.napisat_hui()


ХУИ


In [46]:
path ='C:/Users/user/Downloads/измерения1.gtm'
# path = 'C:/Users/user/measure.gtm'

file = open (path, 'rb', buffering = 0)

text = file.read()
file.close()

text

b'GTM0\x00dT02\x10\x85xVRE\xb3\x00\x00\x051234\x00\tD\xfe\n\x00\t$\xfe\n\x05\t;\xfe\n\x05\tF\xfe\n\x05\t:\xfe\n\x05\t6\xfe\n\x05\t.\xfe\n\x05\t9\xfe\n\x05\t\x13\xfe\n\x05\t3\xfe\n\x05\t$\xfe\n\x05\t\x10\xfe\n\n\t:\xfe\n\n\t$\xfe\n\n\t2\xfe\n\n\t;\xfe\n\n\x00d\x00dT02\x10:\xb2jRE\xb3\x00\x00\x05\xd1\xca\xc21\x00\x08{\xfe\n\x00\x08\x8c\xfe\n\x05\x08\x1d\xfe\n\x05\x08O\xfe\n\x05\x08\x05\xfe\n\x05\x08s\xfe\n\x05\x07\xfe\xfe\n\x05\x08\x8e\xfe\n\x05\x07\xfe\xfe\n\x05\x08\x89\xfe\n\x05\x07\xcd\xfe\n\x05\x07\xd9\xfe\n\n\x08\x17\xfe\n\n\x07\xdd\xfe\n\n\x08#\xfe\n\n\x07\xfa\xfe\n\n\x00d\x00hT02\x10\xa8\x85zRO\xd2\x00\x00\t21\xc0\xc1\xc3\xc2\xc1\xc1\x00\x07t\xfe\n\x00\x07O\xfe\n\x05\x07{\xfe\n\x05\x07\x10\xfe\n\x05\x07\x18\xfe\n\x05\x06\xb3\xfe\n\x05\x06\xcd\xfe\n\x05\x06\xb2\xfe\n\x05\x06i\xfe\n\x05\x06\xa0\xfe\n\x05\x07\x01\xfe\n\x05\x00\xe5\xfe\n\n\xfd\xe7\xfe\n\n\xfd\xe2\xfe\n\n\xfe/\xfe\n\n\xfe6\xfe\n\n\x00h\x00hT02\x10\xd0p|RO\xd2\x00\x00\t21\xc0\xc1\xc3\xc2\xc1\xc1\x00\xff\xdb\xfe\n\x00\xf

In [248]:
data

23.47

In [259]:
data = np.float16(23.47).tostring()
data
# data = 23.47
# print(hex(np.float16(data).tostring()[0]), hex(np.float16(data).tostring()[1]))

b'\xdeM'

In [224]:
hex(np.float16(2.3).tostring()[1])

'0x40'

In [420]:
file = open (path, 'rb', buffering = 0)
text = file.read()
text

b'GTM0\x00cT02\x10\x13YqRO\xc8\x00\x00\x04HUI\x00\t+\xfe\n\x00\x08\xe7\xfe\n\x05\x08\xf0\xfe\n\x05\t\x07\xfe\n\x05\x08\x82\xfe\n\x05\x08\xf0\xfe\n\x05\x08\xd0\xfe\n\x05\x08\xe2\xfe\n\x05\x08\xd2\xfe\n\x05\x08\xe5\xfe\n\x05\t?\xfe\n\x05\tf\xfe\n\n\t\x8d\xfe\n\n\x08\xb3\xfe\n\n\t\xb6\xfe\n\n\x08\xe5\xfe\n\n\x00c\x00cT02\x10\xc1YqRO\xc8\x00\x00\x04Hui\x00\n"\xfe\n\x00\t\xdc\xfe\n\x05\t\xd4\xfe\n\x05\t\xe5\xfe\n\x05\t`\xfe\n\x05\t\xcc\xfe\n\x05\t\x8d\xfe\n\x05\t\xc5\xfe\n\x05\t\xaf\xfe\n\x05\t\xbb\xfe\n\x05\n*\xfe\n\x05\nQ\xfe\n\n\n\x97\xfe\n\n\to\xfe\n\n\n\xa1\xfe\n\n\t\xae\xfe\n\n\x00c\x00cT02\x10\xdbYqRO\xc8\x00\x00\x04Hiy\x00\n<\xfe\n\x00\t\xf5\xfe\n\x05\t\xf4\xfe\n\x05\t\xfe\xfe\n\x05\t\x7f\xfe\n\x05\t\xe5\xfe\n\x05\t\xa7\xfe\n\x05\t\xde\xfe\n\x05\x00\x00\xfe\x00\x05\x00\x00\xfe\x00\x05\x00\x00\xfe\x00\x05\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00\x00\xfe\x00\n\x00c\x00cT02\x10#[qRO\xc8\x00\x00\x04hhh\x00\t\xe2\xfe\n\x00\t\xc2\xfe\n\x05\t\xba\xfe\n\x0

In [28]:
len(coding)

59